In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# set up the driver
driver = webdriver.Firefox()
url = 'https://www.petronas.com/media/reports'
driver.get(url)

# define a function to scroll to the bottom of the page
def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5) # wait for 2 seconds to load more content
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height != last_height:
        scroll_to_bottom() # scroll again if more content is loaded

# scroll to the bottom of the page to load all the reports
scroll_to_bottom()

# extract the URLs of all PDF reports
soup = BeautifulSoup(driver.page_source, 'html.parser')
pdf_urls = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.pdf')]

# print the PDF URLs
print(pdf_urls)

# close the driver
driver.quit()


['/sites/default/files/about-us/petronas-group-of-companies-2021.pdf', '/sites/default/files/about-us/petronas-group-of-companies-2021.pdf', '/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/PETRONAS%20Activity%20Outlook%202023-2025.pdf', '/sites/default/files/uploads/content/2023/PETRONAS%20Activity%20Outlook%202023-2025.pdf', '/sites/default/files/uploads/content/2022/PETRONAS%20Group%20Quarterly%20Report%20Q3%20FY2022.pdf', '/sites/default/files/uploads/content/2022/PETRONAS%20Group%20Quarterly%20Report%20Q3%20FY2022.pdf', '/sites/default/files/uploads/content/2022/Financial_Operational%20Report_Q3%20FY2022.pdf',

In [9]:
import requests
import PyPDF2
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Replace with your own Text Analytics API key and endpoint
key = "0cebfaee5b5a4c3a9c4d504320150da0"
endpoint = "https://cwb-petronas-vivien.cognitiveservices.azure.com/"

# Create a Text Analytics client instance
text_analytics_client = TextAnalyticsClient(endpoint, AzureKeyCredential(key))

# Define the list of URLs that you want to extract text from
url_list = ['/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/Financial%20Operational%20Report%20Q4%20FY2022.pdf', '/sites/default/files/uploads/content/2023/PETRONAS%20Activity%20Outlook%202023-2025.pdf', '/sites/default/files/uploads/content/2023/PETRONAS%20Activity%20Outlook%202023-2025.pdf', '/sites/default/files/uploads/content/2022/PETRONAS%20Group%20Quarterly%20Report%20Q3%20FY2022.pdf', '/sites/default/files/uploads/content/2022/PETRONAS%20Group%20Quarterly%20Report%20Q3%20FY2022.pdf', '/sites/default/files/uploads/content/2022/Financial_Operational%20Report_Q3%20FY2022.pdf', '/sites/default/files/uploads/content/2022/Financial_Operational%20Report_Q3%20FY2022.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2022.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2022.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q2-FY2022.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q2-FY2022.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2022.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2022.pdf', '/sites/default/files/uploads/content/2022/Financial%20Report%20Q1%202022-v2.pdf', '/sites/default/files/uploads/content/2022/Financial%20Report%20Q1%202022-v2.pdf', '/sites/default/files/uploads/content/2022/Redemption%20of%203.125%25%20Guaranteed%20Notes%20Issued%20by%20PETRONAS%20Capital%20Ltd_.pdf', '/sites/default/files/uploads/content/2022/Redemption%20of%203.125%25%20Guaranteed%20Notes%20Issued%20by%20PETRONAS%20Capital%20Ltd_.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Activity-Outlook-2022-2024.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Activity-Outlook-2022-2024.pdf', '/sites/default/files/uploads/content/2023/PETRONAS-PIR2021-Integrated-Report-2021.pdf', '/sites/default/files/uploads/content/2023/PETRONAS-PIR2021-Integrated-Report-2021.pdf', '/sites/default/files/uploads/content/2022/H2-2021-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/H2-2021-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q4-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q4-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q3-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q3-2021.pdf', '/sites/default/files/uploads/content/2022/H1-2021-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/H1-2021-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q2-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q2-2021.pdf', '/sites/default/files/uploads/content/2022/US%241%2C250%2C000%2C000%202.480%25%20Guaranteed%20Notes%20due%202032%20and%20US%241%2C750%2C000%2C000%203.404%25%20Guaranteed%20Notes%20due%202061%20%E2%80%93%20Offering%20Circular%20and%20Pricing%20Supplement.pdf', '/sites/default/files/uploads/content/2022/US%241%2C250%2C000%2C000%202.480%25%20Guaranteed%20Notes%20due%202032%20and%20US%241%2C750%2C000%2C000%203.404%25%20Guaranteed%20Notes%20due%202061%20%E2%80%93%20Offering%20Circular%20and%20Pricing%20Supplement.pdf', '/sites/default/files/uploads/content/2022/Notice%20of%20Listing%20on%20The%20Stock%20Exchange%20of%20Hong%20Kong%20Limited.pdf', '/sites/default/files/uploads/content/2022/Notice%20of%20Listing%20on%20The%20Stock%20Exchange%20of%20Hong%20Kong%20Limited.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q1-2021.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q1-2021.pdf', '/sites/default/files/uploads/content/2022/Cover%20Note%20on%20Announcement%20on%20Publication%20%20-%20PET.pdf', '/sites/default/files/uploads/content/2022/Cover%20Note%20on%20Announcement%20on%20Publication%20%20-%20PET.pdf', '/sites/default/files/uploads/content/2022/Cover%20Note%20for%20Formal%20Notice%20PET.pdf', '/sites/default/files/uploads/content/2022/Cover%20Note%20for%20Formal%20Notice%20PET.pdf', '/sites/default/files/uploads/content/2022/4.%20Cover%20note%20for%20announcement%20on%20publication%20of%20OC.pdf', '/sites/default/files/uploads/content/2022/4.%20Cover%20note%20for%20announcement%20on%20publication%20of%20OC.pdf', '/sites/default/files/uploads/content/2022/2.%20Cover%20note%20for%20formal%20notice%20of%20listing.pdf', '/sites/default/files/uploads/content/2022/2.%20Cover%20note%20for%20formal%20notice%20of%20listing.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Activity-Outlook-2021-2023.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Activity-Outlook-2021-2023.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-IR20-Integrated-Report-2020.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-IR20-Integrated-Report-2020.pdf', '/sites/default/files/uploads/content/2022/H2-2020-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/H2-2020-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/FLOW%20Issue%202%2C%202020.pdf', '/sites/default/files/uploads/content/2022/FLOW%20Issue%202%2C%202020.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q4-2020.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q4-2020.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2020.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2020.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Human-Rights-Report-2020.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Human-Rights-Report-2020.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2020.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2020.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2020.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2020.pdf', '/sites/default/files/uploads/content/2022/WEF%20Stakeholder%20Capitalism%20Metrics%20--%20PETRONAS%20disclosure%20FINAL%20.pdf', '/sites/default/files/uploads/content/2022/WEF%20Stakeholder%20Capitalism%20Metrics%20--%20PETRONAS%20disclosure%20FINAL%20.pdf', '/sites/default/files/uploads/content/2022/H1-2020-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/H1-2020-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2020.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2020.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2020.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2020.pdf', '/sites/default/files/uploads/content/2022/Announcement%20on%20HKEX%20PCL%20GMTN%202020%20Issuance.pdf', '/sites/default/files/uploads/content/2022/Announcement%20on%20HKEX%20PCL%20GMTN%202020%20Issuance.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2020.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2020.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2020.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2020.pdf', '/sites/default/files/uploads/content/2022/Notice%20of%20Renewal%20of%20Listing%20on%20The%20Stock%20Exchange%20of%20Hong%20Kong%20Limited.pdf', '/sites/default/files/uploads/content/2022/Notice%20of%20Renewal%20of%20Listing%20on%20The%20Stock%20Exchange%20of%20Hong%20Kong%20Limited.pdf', '/sites/default/files/uploads/content/2022/Redemption%20of%202.707%20Trust%20Certificates.pdf', '/sites/default/files/uploads/content/2022/Redemption%20of%202.707%20Trust%20Certificates.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Activity-Outlook-2020-2022.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Activity-Outlook-2020-2022.pdf', '/sites/default/files/uploads/content/2022/Flow%20magazine_20200609%20Final.pdf', '/sites/default/files/uploads/content/2022/Flow%20magazine_20200609%20Final.pdf', '/sites/default/files/uploads/content/2022/Industrial%20Skills%20Framework%20Oil%20Gas%20Energy%20First%20Edition%20v2.pdf', '/sites/default/files/uploads/content/2022/Industrial%20Skills%20Framework%20Oil%20Gas%20Energy%20First%20Edition%20v2.pdf', '/sites/default/files/uploads/content/2022/2H-2019-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/2H-2019-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/Flow%2045th%20Anniversary%20Issue.pdf', '/sites/default/files/uploads/content/2022/Flow%2045th%20Anniversary%20Issue.pdf', '/sites/default/files/uploads/content/2022/Financial-report-Q4-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-report-Q4-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2019.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Annual%20Report-2019-v2.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Annual%20Report-2019-v2.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q3-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q3-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2019.pdf', '/sites/default/files/uploads/content/2022/H1-2019-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/H1-2019-Sustainability-Disclosure.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2019.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2019.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2019.pdf', '/sites/default/files/uploads/content/2022/PCML_Infill%20EIA_Response%20to%20ECD%20Comments_V1_26%20March%282%29.pdf', '/sites/default/files/uploads/content/2022/PCML_Infill%20EIA_Response%20to%20ECD%20Comments_V1_26%20March%282%29.pdf', '/sites/default/files/uploads/content/2022/PCML_Drilling_Blocks.pdf', '/sites/default/files/uploads/content/2022/PCML_Drilling_Blocks.pdf', '/sites/default/files/uploads/content/2022/PC%20Myanmar%20%28Hong%20Kong%29%20Limited%20IEE%20for%203D%20Seismic%20Survey%20withing%20the%20Yetagun%20Gas%20Field.pdf', '/sites/default/files/uploads/content/2022/PC%20Myanmar%20%28Hong%20Kong%29%20Limited%20IEE%20for%203D%20Seismic%20Survey%20withing%20the%20Yetagun%20Gas%20Field.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Activity-Outlook-2019-2021.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Activity-Outlook-2019-2021.pdf', '/sites/default/files/uploads/content/2022/FLOW%20final%201%20online.pdf', '/sites/default/files/uploads/content/2022/FLOW%20final%201%20online.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2018.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2018.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Sustainability-Report-2018_0.pdf', '/sites/default/files/uploads/content/2022/PETRONAS-Sustainability-Report-2018_0.pdf', '/sites/default/files/uploads/content/2022/flow-issue-3-2018.pdf', '/sites/default/files/uploads/content/2022/flow-issue-3-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q4-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q4-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2018.pdf', '/sites/default/files/uploads/content/2022/Yetagun%20Exploration%20Drilling%202018%20EIA%20%28reduced%20size%29.pdf', '/sites/default/files/uploads/content/2022/Yetagun%20Exploration%20Drilling%202018%20EIA%20%28reduced%20size%29.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q3-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q3-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2018.pdf', '/sites/default/files/uploads/content/2022/IEE%20for%203D%20Seismic%20Survey%20within%20the%20Yetagun%20Gas%20Field_13%20Sept%202018.pdf', '/sites/default/files/uploads/content/2022/IEE%20for%203D%20Seismic%20Survey%20within%20the%20Yetagun%20Gas%20Field_13%20Sept%202018.pdf', '/sites/default/files/uploads/content/2022/Malaysia%20Tax%20Incentives%20-%20Compilation%20and%20Guide%20for%20OGSE%20Sector%20%28portal%29.pdf', '/sites/default/files/uploads/content/2022/Malaysia%20Tax%20Incentives%20-%20Compilation%20and%20Guide%20for%20OGSE%20Sector%20%28portal%29.pdf', '/sites/default/files/uploads/content/2022/flow-issue-2-2018.pdf', '/sites/default/files/uploads/content/2022/flow-issue-2-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q2-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q2-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2018.pdf', '/sites/default/files/uploads/content/2022/PCMI_Block_IOR7_FINAL%20%28Dec%202018%29.pdf', '/sites/default/files/uploads/content/2022/PCMI_Block_IOR7_FINAL%20%28Dec%202018%29.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2018.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2018.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2018.pdf', '/sites/default/files/uploads/content/2022/petronas-activity-outlook-2018-2020.pdf', '/sites/default/files/uploads/content/2022/petronas-activity-outlook-2018-2020.pdf', '/sites/default/files/uploads/content/2022/flow-issue-1-2018.pdf', '/sites/default/files/uploads/content/2022/flow-issue-1-2018.pdf', '/sites/default/files/uploads/content/2022/sustainability-report-2017.pdf', '/sites/default/files/uploads/content/2022/sustainability-report-2017.pdf', '/sites/default/files/uploads/content/2022/flow-issue-3-2017-revised.pdf', '/sites/default/files/uploads/content/2022/flow-issue-3-2017-revised.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2017.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2017.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2017.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2017.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q4-2017.pdf', '/sites/default/files/uploads/content/2022/Financial-Report-Q4-2017.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2017.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2017.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2017.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2017.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2017.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2017.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2017.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2017.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2017.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2017.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2017.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2017.pdf', '/sites/default/files/uploads/content/2022/flow-issue-1-2017_0.pdf', '/sites/default/files/uploads/content/2022/flow-issue-1-2017_0.pdf', '/sites/default/files/uploads/content/2022/sustainability-report-2016.pdf', '/sites/default/files/uploads/content/2022/sustainability-report-2016.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2016.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2016.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q4-2016.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q4-2016.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2016.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2016.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2016.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2016.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2016.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2016.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2016.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2016.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2016.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2016.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2016.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2016.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2015.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2015.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q4-2015.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q4-2015.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2015.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2015.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2015.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2015.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2015.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2015.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2015.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2015.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2015.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2015.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2015.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2015.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2015.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2015.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2015.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2015.pdf', '/sites/default/files/uploads/content/2022/pcmi-block-IOR7-jan-2015.pdf', '/sites/default/files/uploads/content/2022/pcmi-block-IOR7-jan-2015.pdf', '/sites/default/files/uploads/content/2022/pcmi-block-IOR5.pdf', '/sites/default/files/uploads/content/2022/pcmi-block-IOR5.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2014.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2014.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q4-2014.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q4-2014.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2014.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2014.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2014.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2014.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2014.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2014.pdf', '/sites/default/files/uploads/content/2022/financial-operational-report-third-quarter-30-september-2014-fy2014.pdf', '/sites/default/files/uploads/content/2022/financial-operational-report-third-quarter-30-september-2014-fy2014.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2014.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2014.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2014_0.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2014_0.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2014.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2014.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2014.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2014.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2013.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2013.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q4-2013.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q4-2013.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2013.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2013.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2013.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2013.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2013.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2013.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2013.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2013.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2013.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2013.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2013.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2013.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2013.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2013.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2013.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2013.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2012.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2012.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q4-2012.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q4-2012.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2012.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q4-2012.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2012.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2012.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2012.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q3-2012.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2012.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q3-2012.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2012.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q2-2012.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2012.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q2-2012.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2012.pdf', '/sites/default/files/uploads/content/2022/Interim-Financial-Report-Q1-2012.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2012.pdf', '/sites/default/files/uploads/content/2022/Financial-Operational-Report-Q1-2012.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2011.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2011.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2011.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2011.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2010.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2010.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2010.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2010.pdf', '/sites/default/files/uploads/content/2022/sustainability-report-2009.pdf', '/sites/default/files/uploads/content/2022/sustainability-report-2009.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2009.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2009.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2008.pdf', '/sites/default/files/uploads/content/2022/petronas-annual-report-2008.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2007.pdf', '/sites/default/files/uploads/content/2022/Sustainability-Report-2007.pdf']

# Loop through each URL and extract the text content using PyPDF2
for url in url_list:
    # Download the PDF from the URL
    response = requests.get(url)
    
    # Extract the text content from the PDF
    pdf_file = PyPDF2.PdfFileReader(io.BytesIO(response.content))
    text = ""
    for page_num in range(pdf_file.getNumPages()):
        page = pdf_file.getPage(page_num)
        text += page.extractText()
        
    # Analyze the extracted text using the Text Analytics service
    result = text_analytics_client.analyze_sentiment(text, language="en")
    
    # Print the sentiment score and sentiment label for the text
    print(f"Sentiment score: {result.sentiment_scores}")
    print(f"Sentiment label: {result.sentiment}")


MissingSchema: Invalid URL '/sites/default/files/about-us/petronas-group-of-companies-2021.pdf': No scheme supplied. Perhaps you meant https:///sites/default/files/about-us/petronas-group-of-companies-2021.pdf?